In [ ]:
!pip install pandas
!pip install optuna
!pip install optuna-dashboard
!pip install scikit
!pip install imbalanced-learn
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


In [26]:
random_state = 42
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])
# print(X.describe())
# X.drop(columns = ['Age', 'Gender', 'UAlb', 'Ucr', 'UACR', 'LDLC', 'HDLC'], inplace=True)

# [Age,Gender,UAlb,Ucr,UACR,TC,TG,TCTG,LDLC,HDLC,Scr,BUN,FPG,HbA1c,Height,Weight,BMI,Duration,DR,Community_baihe,Community_chonggu,Community_huaxin,Community_jinze,Community_liantang,Community_xianghuaqiao,Community_xujin,Community_yingpu,Community_zhaoxian,Community_zhujiajiao]
#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)

Using device: cuda


In [27]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from modularModels1 import BlockMaker, modularNN, BasicModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, drop_last=True)
    return train_loader, val_loader 

def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

Using cuda


In [28]:
from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None, alpha:float=None, gamma:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss(alpha =alpha, gamma=gamma)
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def augment_data_in_place(X, X_test, normalisation_method=MinMaxScaler()):
    all_numerical_columns = [
        'Age', 'Height', 'Weight', 'Duration',
        'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 
        'TCTG', 'LDLC', 'HDLC', 'Scr', 'BUN', 'FPG', 'HbA1c'
    ]

    # Find which of those columns actually exist in both X and X_test
    existing_columns = [col for col in all_numerical_columns if col in X.columns and col in X_test.columns]
    
    if not existing_columns:
        print("No matching columns found for augmentation. Normalised data only.")
        X = normalisation_method.fit_transform(X)
        X_test = normalisation_method.transform(X_test)
        return X, X_test

    # 1. Log-transform
    X_copy = X.copy()
    X_test_copy = X_test.copy()
    X_copy.loc[:, existing_columns] = X_copy.loc[:, existing_columns].apply(np.log1p)
    X_test_copy.loc[:, existing_columns] = X_test_copy.loc[:, existing_columns].apply(np.log1p)

    # 2. Add Gaussian noise to training data only
    noise = np.random.normal(0, 0.15, X_copy[existing_columns].shape)
    X_copy.loc[:, existing_columns] = X_copy.loc[:, existing_columns] + noise

    # 3. Fit scaler on train, transform both
    scaler = normalisation_method
    X_copy.loc[:, existing_columns] = scaler.fit_transform(X_copy.loc[:, existing_columns])
    X_test_copy.loc[:, existing_columns] = scaler.transform(X_test_copy.loc[:, existing_columns])

    return X_copy, X_test_copy


In [43]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None, contamination=0.05):
    """
    Generates stratified folds with specified normalization.

    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    - MaxAbsScaler()
    - QuantileTransformer(output_distribution='uniform')

    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []

    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        # print("Original\n", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
        X_train_cleaned, Y_train_cleaned = X_train.copy(), Y_train.copy()
        if contamination is not None and contamination > 0: #? using contamination = 0.0 works
            X_train_zeros = X_train[Y_train.iloc[:, 0] == 0]
            X_train_ones = X_train[Y_train.iloc[:, 0] == 1]
            Y_train_zeros = Y_train[Y_train.iloc[:, 0] == 0]
            Y_train_ones = Y_train[Y_train.iloc[:, 0] == 1] 
            # print("Ones and zeros\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
            #only class 0s
            if X_train_zeros.isna().any().any():
                print("got NaN values in the training set")
            
            # Apply Isolation Forest to majority class only
            iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
            try:
                outliers = iso_forest.fit_predict(X_train_zeros)
            except UserWarning as e:
                print("Caught warning during IsolationForest fitting:", e)
                outliers = np.ones(len(X_train_zeros))  # If warning occurs, keep all data
            # Keep only non-outlier majority samples
            X_train_zeros = X_train_zeros[outliers == 1]
            Y_train_zeros = Y_train_zeros[outliers == 1]
            # print("After iso:\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
            
            # Combine the cleaned majority class with the untouched minority class
            X_train_cleaned = pd.concat([X_train_zeros, X_train_ones])
            Y_train_cleaned = pd.concat([Y_train_zeros, Y_train_ones])
        #? data augmentation on leftover data
        X_train_scaled, X_test_scaled = augment_data_in_place(X_train_cleaned, X_test, normalisation_method=normalisation_method)
        
        # Handle oversampling if needed
        #! use X_train_scaled and Y_train_cleaned for oversampling becasue y_train_cleaned no changes after augmentation
        if oversampler:
            X_train_scaled, Y_train_cleaned = oversampler.fit_resample(X_train_scaled, Y_train_cleaned)

        # Convert scaled data back to DataFrame with the correct column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_cleaned.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

        # Handle community columns
        community_cols = [col for col in X_train_scaled.columns if col.startswith('Community')]
        if community_cols:
            X_train_scaled[community_cols] = X_train_scaled[community_cols].apply(
                lambda row: pd.Series(np.eye(len(row))[row.argmax()]), axis=1
            ).set_axis(community_cols, axis=1)
        # print(X_train_scaled[community_cols].describe())

        # Ensure 'Gender' is still binary (0 or 1)
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)

        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train_cleaned, Y_test))

        print(f"Fold: {fold+1}, Train: {X_train_scaled.shape}, Test: {X_test_scaled.shape}")

    return kFolds_list

In [44]:
# oversampler = None
# contamination = 0.05
# normalisation_method = QuantileTransformer()
# kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
#                          normalisation_method = normalisation_method, 
#                          n_splits=5, 
#                          oversampler = oversampler, random_state=42, contamination=contamination)

In [45]:
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device, threshold = 0.5):
    if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
        raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0

    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            outputs = model(inputs) #? Forward pass through the model
            loss = criterion(outputs, labels) #? Calculate loss
            loss.backward() #? Backpropagation
            running_loss += loss.item()
            optimiser.step() #? Update weights
            scheduler.step()
        # if scheduler: 
        #     if not isinstance(scheduler,torch.optim.lr_scheduler.OneCycleLR):
        #         scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
            avg_val_loss = val_loss / len(val_loader)
            
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        elif avg_val_loss*0.95 <= best_val_loss:
                wait = 0
        else:
            wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
        print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.3f}".ljust(16), f"best_val_loss:{best_val_loss:.3f}".ljust(12), f"Val Loss: {avg_val_loss:.3f}", f"Scheduler lr: {scheduler.get_last_lr()}",end="\r")
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                # predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                predictions = (torch.sigmoid(outputs) >= threshold).float().cpu().numpy()
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc


In [46]:
import torch
import torch.nn as nn

class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=None, activation=nn.ReLU):
        super().__init__()
        layers = [
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            activation()
        ]
        if dropout and dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)


class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout= None, hidden_dim2= None):
        super().__init__()
        
        # A couple of FeedForward blocks
        self.block1 = FeedForwardBlock(input_dim, hidden_dim, dropout)
        self.block2 = FeedForwardBlock(hidden_dim, output_dim, dropout)
        # self.block3 = FeedForwardBlock(hidden_dim2, output_dim, dropout = None)

        # Final output layer (could be softmax, sigmoid, or whatever your target is)
        self.output_layer = nn.Linear(output_dim, 1)  # Just in case you're doing regression or binary classification

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        # x = self.block3(x)
        x = self.output_layer(x)  # Final linear layer
        return x
    
    def last_layer(self):
        return self.output_layer
test_model = MyModel(8,64,16,0)
print(test_model)


MyModel(
  (block1): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (block2): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=64, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (output_layer): Linear(in_features=16, out_features=1, bias=True)
)


In [50]:
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 10000
    random_state = 42

    n_neighbours = trial.suggest_int("n_neighbours", 2, 10)
    oversampler = ADASYN(sampling_strategy='minority', n_neighbors=n_neighbours, random_state=random_state)
    
    normalisation_method = trial.suggest_categorical("normalisation_method", ["MinMaxScaler",])
    if normalisation_method:
        if normalisation_method == "MinMaxScaler":
            normalisation_method = MinMaxScaler()
        else:
            normalisation_method = MinMaxScaler()
    contamination = trial.suggest_float("contamination", 0.19, 0.4)
    kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42, contamination=contamination)
                        
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 64, 128)
    # hidden_dim2 = trial.suggest_int("hidden_dim2", 20, 100)
    hidden_dim2 = None
    output_dim = trial.suggest_int("output_dim", 8, hidden_dim)
    
    dropout = trial.suggest_float("dropout", 0.00, 0.00)
    threshold = trial.suggest_float("threshold", 0.3, 0.8)
    # dropout = None
    initial_lr = trial.suggest_float("initial_lr", 1e-4, 1e-3, log=True)
    max_lr = trial.suggest_float("max_lr", 5e-3, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    
    # Loss function hyperparameters
    criterion_choice = trial.suggest_categorical("criterion", ["FocalLoss"]) 
    # criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss"]) 
    
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 10)
        alpha = None
        gamma = None
    elif criterion_choice == "FocalLoss":
        pos_weight= None
        alpha = trial.suggest_float("alpha", .8, 2)
        gamma = trial.suggest_float("gamma", 0.1, 1.5)
    else:
        pos_weight = None
    
    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        print(f"Fold {fold}:")
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device)
        # Instantiate and initialize the model
        model = MyModel(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, output_dim=hidden_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)
        
        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimiser, base_lr=1e-5, max_lr=5e-4)
        
        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=10, device=device, threshold = threshold
        )
        print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")
        del model
        del train_loader
        del val_loader
    
        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score


In [51]:
# import threading
# import optuna
# from optuna_dashboard import run_server

# def start_dashboard():
#     run_server(storage)

# storage = optuna.storages.InMemoryStorage()
# study = optuna.create_study(direction="maximize", storage=storage, study_name="Basic")

# # Start dashboard in a separate thread
# dashboard_thread = threading.Thread(target=start_dashboard, daemon=True)
# dashboard_thread.start()

# # Run optimization
# study.optimize(maximise_combined_score, n_trials=30)

# # After optimization, print results
# print("Best trial:")
# trial = study.best_trial
# print(f"  Combined score: {trial.value}")
# print("  Best hyperparameters:")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")


In [52]:
import threading
import optuna
from optuna_dashboard import run_server
# !fuser -k 8080/tcp

# Define your persistent storage
storage = "sqlite:///optana5.db"

# Create or load your study
study_name = "optuna5"
try:
    study = optuna.load_study(study_name=study_name, storage=storage)
except KeyError:
    study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage)

# Start Optuna Dashboard in a separate thread
dashboard_thread = threading.Thread(target=lambda: run_server(storage), daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=100)

# Print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


Using device: cuda


Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



Fold: 1, Train: (6004, 28), Test: (1149, 28)
Fold: 2, Train: (5849, 28), Test: (1149, 28)
Fold: 3, Train: (6005, 28), Test: (1148, 28)
Fold: 4, Train: (6030, 28), Test: (1148, 28)
Fold: 5, Train: (6015, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 38, best val loss: 0.7317ss: 0.922 Scheduler lr: [0.00014695500000000005]
Accuracy: 0.7137, precision: 0.1858, recall: 0.5431, f1: 0.2769, auc: 0.6380
Fold 2:
Early stopping triggered at epoch 37, best val loss: 0.7510ss: 1.063 Scheduler lr: [0.00018738000000000004]
Accuracy: 0.7415, precision: 0.1953, recall: 0.5000, f1: 0.2809, auc: 0.6343
Fold 3:
Early stopping triggered at epoch 12, best val loss: 0.6210ss: 0.896 Scheduler lr: [0.00026063499999999995]
Accuracy: 0.7317, precision: 0.2055, recall: 0.5776, f1: 0.3032, auc: 0.6633
Fold 4:
Early stopping triggered at epoch 26, best val loss: 0.7499ss: 1.055 Scheduler lr: [0.0004142500000000001]]
Accuracy: 0.6812, precision: 0.1827, recall: 0.6207, f1: 0.2824, auc: 0.6543
Fol

[I 2025-04-15 16:22:59,154] Trial 6 finished with value: 0.2875099404123068 and parameters: {'n_neighbours': 6, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.2689168150496098, 'hidden_dim': 77, 'output_dim': 59, 'dropout': 0.0, 'threshold': 0.5169466057355765, 'initial_lr': 0.0003188097133591157, 'max_lr': 0.005, 'weight_decay': 4.3770671509337525e-05, 'criterion': 'FocalLoss', 'alpha': 1.9294705181109608, 'gamma': 0.37194642919416054}. Best is trial 3 with value: 0.2881741376825723.


Early stopping triggered at epoch 24, best val loss: 0.7884
Accuracy: 0.6908, precision: 0.1912, recall: 0.6379, f1: 0.2942, auc: 0.6673
Using device: cuda
Fold: 1, Train: (5589, 28), Test: (1149, 28)
Fold: 2, Train: (5594, 28), Test: (1149, 28)
Fold: 3, Train: (5613, 28), Test: (1148, 28)
Fold: 4, Train: (5605, 28), Test: (1148, 28)
Fold: 5, Train: (5623, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 40, best val loss: 0.3499ss: 0.489 Scheduler lr: [0.00015871500000000006]
Accuracy: 0.5361, precision: 0.1508, recall: 0.7759, f1: 0.2525, auc: 0.6425
Fold 2:
Early stopping triggered at epoch 26, best val loss: 0.4032ss: 0.471 Scheduler lr: [0.00045712500000000007]
Accuracy: 0.4552, precision: 0.1429, recall: 0.8793, f1: 0.2458, auc: 0.6434
Fold 3:
Early stopping triggered at epoch 33, best val loss: 0.3163ss: 0.700 Scheduler lr: [0.00030792000000000005]
Accuracy: 0.5627, precision: 0.1541, recall: 0.7414, f1: 0.2552, auc: 0.6420
Fold 4:
Early stopping triggered at epoc

...

Using device: cuda
Fold: 1, Train: (4921, 28), Test: (1149, 28)
Fold: 2, Train: (4916, 28), Test: (1149, 28)
Fold: 3, Train: (4926, 28), Test: (1148, 28)
Fold: 4, Train: (4908, 28), Test: (1148, 28)
Fold: 5, Train: (4898, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 17, best val loss: 0.3835ss: 0.503 Scheduler lr: [0.00030792000000000005]
Accuracy: 0.7572, precision: 0.2140, recall: 0.5259, f1: 0.3042, auc: 0.6545
Fold 2:
Early stopping triggered at epoch 20, best val loss: 0.4187ss: 0.506 Scheduler lr: [0.00036378]0000000001]]
Accuracy: 0.7433, precision: 0.1837, recall: 0.4483, f1: 0.2607, auc: 0.6123
Fold 3:
Early stopping triggered at epoch 43, best val loss: 0.3153ss: 0.565 Scheduler lr: [0.00020795999999999995]
Accuracy: 0.7953, precision: 0.2308, recall: 0.4397, f1: 0.3027, auc: 0.6375
Fold 4:
Early stopping triggered at epoch 39, best val loss: 0.2840ss: 0.645 Scheduler lr: [0.00028244000000000007]
Accuracy: 0.7448, precision: 0.2117, recall: 0.5603, f1: 0.30

[I 2025-04-15 17:22:09,517] Trial 104 finished with value: 0.2913749186237885 and parameters: {'n_neighbours': 4, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.39607510491121795, 'hidden_dim': 77, 'output_dim': 74, 'dropout': 0.0, 'threshold': 0.6047475887920116, 'initial_lr': 0.00024621775591777896, 'max_lr': 0.005, 'weight_decay': 0.0076264759145588815, 'criterion': 'FocalLoss', 'alpha': 1.0874552687507697, 'gamma': 0.8063129135203743}. Best is trial 93 with value: 0.31120562348708986.


Early stopping triggered at epoch 45, best val loss: 0.2873
Accuracy: 0.7605, precision: 0.2022, recall: 0.4655, f1: 0.2820, auc: 0.6296
Using device: cuda
Fold: 1, Train: (5048, 28), Test: (1149, 28)
Fold: 2, Train: (5029, 28), Test: (1149, 28)
Fold: 3, Train: (5229, 28), Test: (1148, 28)
Fold: 4, Train: (5020, 28), Test: (1148, 28)
Fold: 5, Train: (5251, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 13, best val loss: 0.2972ss: 0.335 Scheduler lr: [0.00023931999999999998]
Accuracy: 0.8329, precision: 0.2625, recall: 0.3621, f1: 0.3043, auc: 0.6239
Fold 2:
Early stopping triggered at epoch 26, best val loss: 0.3050ss: 0.409 Scheduler lr: [0.00048775000000000007]
Accuracy: 0.7232, precision: 0.1783, recall: 0.4828, f1: 0.2605, auc: 0.6165
Fold 3:
Early stopping triggered at epoch 35, best val loss: 0.2711ss: 0.331 Scheduler lr: [0.00031527000000000014]
Accuracy: 0.8057, precision: 0.2312, recall: 0.3966, f1: 0.2921, auc: 0.6241
Fold 4:
Early stopping triggered at epoc

[I 2025-04-15 17:22:45,293] Trial 105 finished with value: 0.2928229248522579 and parameters: {'n_neighbours': 3, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.3841524246822607, 'hidden_dim': 67, 'output_dim': 65, 'dropout': 0.0, 'threshold': 0.653659086553296, 'initial_lr': 0.0002216627168728275, 'max_lr': 0.005, 'weight_decay': 0.0089466635005589, 'criterion': 'FocalLoss', 'alpha': 0.945539681462489, 'gamma': 0.8560208391871713}. Best is trial 93 with value: 0.31120562348708986.


Accuracy: 0.7840, precision: 0.2250, recall: 0.4655, f1: 0.3034, auc: 0.6426
Best trial:
  Combined score: 0.31120562348708986
  Best hyperparameters:
    n_neighbours: 6
    normalisation_method: MinMaxScaler
    contamination: 0.3908182381776086
    hidden_dim: 68
    output_dim: 66
    dropout: 0.0
    threshold: 0.555927695059721
    initial_lr: 0.0002473479892457255
    max_lr: 0.005
    weight_decay: 0.006571168608747022
    criterion: FocalLoss
    alpha: 0.908287944147098
    gamma: 0.8361059696703981
